# 利用 SVD 简化数据

内容安排
1. SVD 概念和基于 SVD 的简化数据
2. 基于协同过滤的推荐系统以及利用 SVD 提高推荐效果
3. 基于 SVD 的图像压缩


## 1. SVD 概念

**SVD**：全称为奇异值分解。

**SVD 性质**：对于奇异值,它跟我们特征分解中的特征值类似，在奇异值矩阵中也是按照从大到小排列，而且奇异值的减少特别的快，在很多情况下，前10%甚至1%的奇异值的和就占了全部的奇异值之和的99%以上的比例。也就是说，我们也可以用最大的k个的奇异值和对应的左右奇异向量来近似描述矩阵。也就是说：$$A_{m×n}=U_{m×m}Σ_{m×n}V^T_{n×n}≈U_{m×k}Σ_{k×k}V^T_{k×n}$$ 其中 $k$ 要比 $n$ 小很多，也就是一个大的矩阵 $A$ 可以用三个小的矩阵$U_{m×k},Σ_{k×k},V^T_{k×n}$ 来表示。

**SVD 作用**：由于上述的性质，
1. 可以用于PCA降维，来做数据压缩和去噪。*（PS：实际上，scikit-learn的PCA算法的背后真正的实现就是用的SVD，而不是我们我们认为的暴力特征分解。）*
2. 可以用于推荐算法，将用户和喜好对应的矩阵做特征分解，进而得到隐含的用户需求来做推荐
3. 可以用于NLP中的算法，比如潜在语义索引（LSI）

**SVD 缺点**：分解出的矩阵解释性往往不强，有点黑盒子的味道

## 2. 基于协同过滤的推荐系统以及利用 SVD 提高推荐效果

### 2.1 协同过滤

**协同过滤**：是通过用户-物品评分矩阵来计算相似度的。举个例子，评分矩阵如下：

|（评分矩阵）|物品1|物品2|物品3|物品4|物品5|
|--|--|--|--|--|
|用户1|2|0|0|4|4|
|用户2|5|5|5|3|3|
|用户3|2|4|2|1|2|

### 2.2 相似度计算

**相似度的计算公式**：常用的有 3 种，计算向量 $v_1,v_2$ 的相似度
1. 欧氏距离法。$sim(v_1,v_2) = \frac{1}{1+dist(v_1,v_2)}$，其中 $dist(v_1,v_2)$ 表示两个向量的欧氏距离。
2. 皮尔逊相关系数法。$sim(v_1,v_2) = 0.5 + 0.5*corrcoef(v_1,v_2)$，其中 $corrcoef(v_1,v_2)$ 表示两个向量的皮尔逊相关系数。
3. 余弦相似度法。其计算的是两个向量夹角的余弦值。$sim(v_1,v_2) = 0.5 + 0.5*cos\theta = 0.5 + 0.5*\frac{v_1 \cdot v_2}{||v_1|| \cdot ||v_2||}$，其中 $||A||$ 称为向量 A 的 2 范数（通俗理解就是 A 点到原点的距离），如 $||(4,2,2)||=\sqrt{4^2+2^2+2^2}$

其中，上面的公式得到的相似度取值范围 [0,1]

**用户间相似度的计算：评分矩阵种每行表示为一个用户向量。**以上表为例子，用户 1 向量表示为 (2,0,0,4,4)，用户 2 向量表示为 (5,5,5,3,3)。套用上面 3 个公式其中之一就能计算出两个用户的相似度

**物品间相似度的计算：评分矩阵种每列表示为一个物品向量。**以上表为例子，物品 1 向量表示为 (2,5,2)，物品 2 向量表示为 (0,5,4)

In [16]:
import numpy as np

def sim_euclid(v1, v2):
    '''
    欧氏距离法计算两个向量的相似度
    :param v1: [np.array(m,n)] 
    :param v2: [np.array(m,n)]
    :return : [float] sim 相似度
    '''
    assert(v1.shape == v2.shape)
    return 1.0/(1.0+np.linalg.norm(v1-v2))

def sim_pear(v1, v2):
    '''
    皮尔逊相关系数法计算两个向量的相似度
    :param v1: [np.array(m,n)] 
    :param v2: [np.array(m,n)]
    :return : [float] sim 相似度
    '''
    assert(v1.shape == v2.shape)
    if len(v1)<3:
        return 1.0
    return 0.5+0.5*np.corrcoef(v1, v2, rowvar=0)[0][1]

def sim_cos(v1, v2):
    '''
    余弦相似度法计算两个向量的相似度
    :param v1: [np.array(m,n)] 
    :param v2: [np.array(m,n)]
    :return : [float] sim 相似度
    '''
    assert(v1.shape == v2.shape)
    a = np.dot(v1.T, v2)
    b = np.linalg.norm(v1)*np.linalg.norm(v2)
    return 0.5+0.5*a/b

dataset = np.array([
    [1,1,1,0,0],
    [2,2,2,0,0],
    [1,1,1,0,0],
    [5,5,5,0,0],
    [1,1,0,2,2],
    [0,0,0,3,3],
    [0,0,0,1,1]
])
print(sim_euclid(dataset[:,0], dataset[:,4]),sim_euclid(dataset[:,0], dataset[:,0]))
print(sim_pear(dataset[:,0], dataset[:,4]), sim_pear(dataset[:,0], dataset[:,0]))
print(sim_cos(dataset[:,0].reshape((dataset.shape[0],1)), dataset[:,4].reshape((dataset.shape[0],1))), 
      sim_cos(dataset[:,0].reshape((dataset.shape[0],1)), dataset[:,0].reshape((dataset.shape[0],1))))

0.13367660240019172 1.0
0.23768619407595815 1.0
0.5472455591261534 0.9999999999999999


### 2.3 基于物品相似度的协同过滤

为某个用户推荐他未评分的物品。未评分在评分矩阵用 0 表示。

**推荐的工作过程**：参考 `recommend()`函数
```
找出所有的该用户没有评分的物品
对每个该用户未评分的物品：
    预测该用户对物品的评分
选择预测评分中最高的 k 个物品推荐给该用户
```

**预测该用户对某个未评分过的物品的评分的算法思想**：基于物品的相似度，参考 `estimate_standard()` 函数
```
遍历每个该用户评分过的物品：
    找到对这两个物品都有评分的用户
    将用户对这两个物品的评分组成这两个物品的向量，并计算它们的相似度
    累加相似度
    加权累计相似度
该物品对于该用户的评分等于加权累计相似度/累加相似度
```

In [35]:
def estimate_standard(dataset, user, item, sim_method):
    '''
    未使用 SVD 的预测用户 user 对物品 item 的评分
    :param dataset: [np.array(m,n)] 评分矩阵
    :param user: [int] 用户
    :param item: [int] 物品
    :param sim_method: [function] 相似度计算公式
    '''
    n = dataset.shape[1]
    total_sim = 0.0
    rate_total_sim = 0.0
    for j in range(n):  # 遍历每个物品
        rating = dataset[user, j]
        # 1. 跳过没评分的物品，即遍历每个该用户评分过的物品
        if rating == 0: 
            continue
        # 2. 找到对这两个物品（j,item）都有评分的用户
        user_list = np.nonzero(np.logical_and(dataset[:,item] > 0, dataset[:,j] > 0))
        # 3. 将用户对这两个物品的评分组成这两个物品的向量，并计算它们的相似度
        sim = sim_method(dataset[user_list, item], dataset[user_list, j]) if len(user_list) > 0 else 0
        # 4. 累加相似度
        total_sim += sim
        # 5. 加权累加相似度
        rate_total_sim += sim*rating
    if total_sim == 0:
        return 0
    return rate_total_sim/total_sim

def recommend(dataset, user, N=3, sim_method=sim_euclid, est_method=estimate_standard):
    '''
    向用户推荐 N 个未评分过的物品
    :param dataset: [np.array(m,n)] 评分矩阵
    :param user: [int] 用户
    :param N: [int] 推荐 top N
    :param sim_method: [function] 相似度计算公式
    :param est_method: [function] 预测评分函数
    :return : [list] 
    '''
    # 1. 找出所有的该用户没有评分的物品
    unrated_items = np.nonzero(dataset[user, :] == 0)[0]
    if len(unrated_items) == 0:
        return 'you rated everything'
    items_rate = []
    # 2. 对每个该用户未评分的物品：
    for item in unrated_items:
        # 3. 预测该用户对物品的评分
        predict_rate = est_method(dataset, user, item, sim_method)
        items_rate.append((item, predict_rate))
    # 4. 选择预测评分中最高的 k 个物品推荐给该用户
    print(sorted(items_rate, key=lambda k:k[1], reverse=True))
    return sorted(items_rate, key=lambda k:k[1], reverse=True)[:N]

dataset = np.array([
    [4,4,0,2,2],
    [4,0,0,3,3],
    [4,0,0,1,1],
    [1,1,1,2,0],
    [2,2,2,0,0],
    [1,1,1,0,0],
    [5,0,5,0,0]
])

print(recommend(dataset, 2))

[(1, 2.8266504712098603), (2, 2.2)]
[(1, 2.8266504712098603), (2, 2.2)]


### 2.4 利用 SVD 提高推荐效果

与上面的唯一不同在于在预测评分时物品间的相似度的计算是低维空间下进行的。

**利用 SVD 进行降维的原理**：
1. 将评分矩阵根据 SVD 矩阵分解公式，得到三个矩阵 $U_{m×m},~Σ_{m×n},~V^T_{n×n}$，其中 $Σ$ 存放着奇异值，并在实际中以 $(n,)$ 形式存储，奇异值逐行递减
2. 找到到底有多少个奇异值之和达到总奇异值的 90%（或以上）
3. 假设有 N（N < n） 个，那么利用 SVD 将所有的物品映射到一个低维空间中，计算物品间的相似度



In [40]:
dataset = np.array([
    [2,0,0,4,4,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,5],
    [0,0,0,0,0,0,0,1,0,4,0],
    [3,3,4,0,3,0,0,2,2,0,0],
    [5,5,5,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,5,0,0,5,0],
    [4,0,4,0,0,0,0,0,0,0,5],
    [0,0,0,0,0,0,0,0,0,0,5],
    [0,0,0,0,0,0,0,0,0,0,4],
    [0,0,0,0,0,0,5,0,0,5,0],
    [0,0,0,3,0,0,0,0,4,5,0],
    [1,1,2,1,1,2,1,0,4,5,0]
])
# 找到 N
_,sigma,_ = np.linalg.svd(dataset)
sigma = sigma**2
print(np.sum(sigma[:5] / np.sum(sigma)))
N = 5

# def estimate_svd(dataset, user, item, sim_method, N=5):
#     '''
#     使用 SVD 的预测用户 user 对物品 item 的评分
#     :param dataset: [np.array(m,n)] 评分矩阵
#     :param user: [int] 用户
#     :param item: [int] 物品
#     :param sim_method: [function] 相似度计算公式
#     :parm N: [int] N 个奇异值之和达到总奇异值的 90%（或以上）
#     '''
#     n = dataset.shape[1]
#     total_sim = 0.0
#     rate_total_sim = 0.0
    
#     # 使用 SVD 后的变化 
#     u, sigma, vt = np.linalg.svd(dataset)  # SVD 矩阵分解
#     sig_N = np.array(np.dot(np.eye(N)*sigma[:N]))
#     xformed_items = np.dot(dataset.T, np.dot(u[:,:N], np.linalg.inv(sig_N)))  # 利用矩阵 U 将物品转换到低维空间中
    
#     for j in range(n):  # 遍历每个物品
#         rating = dataset[user, j]
#         # 1. 跳过没评分的物品，即遍历每个该用户评分过的物品
#         if rating == 0: 
#             continue
#         # 2. 找到对这两个物品（j,item）都有评分的用户
#         user_list = np.nonzero(np.logical_and(dataset[:,item] > 0, dataset[:,j] > 0))
#         # 3. 将用户对这两个物品的评分组成这两个物品的向量，并计算它们的相似度
# #         sim = sim_method(dataset[user_list, item], dataset[user_list, j]) if len(user_list) > 0 else 0
#         sim = sim_method(xformed_items[item,:].T, xformed_items[j,:].T)
#         # 4. 累加相似度
#         total_sim += sim
#         # 5. 加权累加相似度
#         rate_total_sim += sim*rating
#     if total_sim == 0:
#         return 0
#     return rate_total_sim/total_sim

# print(recommend(dataset, 1, est_method=estimate_svd))

0.9514904858675788


## 3. 基于 SVD 的图像压缩

